# Entropía condicional

#### Cargamos el texto para su normalización (preprocesamiento).

In [1]:
nombre_archivo = "e960401_mod.htm"
dir_archivo = "./texto/"
archivo = open(dir_archivo + nombre_archivo, encoding='utf-8')
texto = archivo.read()

#### Quitamos las etiquetas html del texto.

In [2]:
def quitar_html(texto):
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(texto, 'lxml')
    return soup.get_text()
texto = quitar_html(texto)

#### Ahora podemos dividir el texto en oraciones.

In [3]:
def obtener_oraciones(texto):
    import nltk
    sent_tokenizer = nltk.data.load('nltk:tokenizers/punkt/english.pickle')
    return sent_tokenizer.tokenize(texto)
oraciones = obtener_oraciones(texto)

#### Ya que tenemos las oraciones hay que realizar varios pasos de normalización. Primero tokenizamos cada oracion. 

In [4]:
def tokenizar_oracion(oracion):
    from nltk.tokenize import word_tokenize
    return word_tokenize(oracion)
oraciones_tokenizadas = [tokenizar_oracion(o) for o in oraciones]

#### Ahora tenemos que etiquetar las partes de cada oracion (POS tagging) con un etiquetador que ya entrenamos. Cargamos el etiquetador.

In [5]:
def cargar_archivo_pickle(nombre):
    import pickle
    with open("./archivo/" + nombre + '.pkl', "rb") as fp:
        arc = pickle.load(fp)
    return arc
eti = cargar_archivo_pickle("tagger")

#### Ahora podemos etiquetar las partes de las oraciones, el etiquetador regresa una lista de tuplas donde el primer elemento es la palabra y el segundo es la parte de la oración que ocupa la palabra.

In [6]:
def etiquetar_oracion(oracion, etiquetador):
    return etiquetador.tag(oracion)
oraciones_etiquetadas = [etiquetar_oracion(o, eti) for o in oraciones_tokenizadas]

#### Ahora tenemos el texto sin etiquetas html, segmentado en oraciones tokenizadas y con cada oración etiquetada, sin embargo, son necesarios más pasos. En este paso vamos a pasar a minúsculas tanto la palabra, como la parte de oración que le corresponde.

In [7]:
def oracion_a_minuscula(oracion):
    return [(tpl[0].lower(), tpl[1][0].lower()) for tpl in oracion]
oraciones_etiquetadas = [oracion_a_minuscula(o) for o in oraciones_etiquetadas]


#### Ahora que las oraciones están etiquetadas y en minúsculas, podemos eliminar cualquier signo de puntación, número o caracter especial del texto.

In [8]:
def quitar_car_especiales(oracion):
    import re
    oracion_limpia = []
    for tpl in oracion:
        cadena_limpia = ""
        for c in tpl[0]:
            if re.match(r'[a-záéíóúñü]', c):
                cadena_limpia += c
        if cadena_limpia != '':
            oracion_limpia.append((cadena_limpia, tpl[1]))
    return oracion_limpia
oraciones_etiquetadas = [quitar_car_especiales(o) for o in oraciones_etiquetadas]

#### Por último quitaremos de cada oración las "stopwords" que son palabras tan comunes que no agregan significado a las oraciones ni al texto en general.

In [9]:
def quitar_stopwords(oracion):
    from nltk.corpus import stopwords
    stopwords_espanol = set(stopwords.words('spanish'))
    return [tpl for tpl in oracion if tpl[0] not in stopwords_espanol]
oraciones_etiquetadas = [quitar_stopwords(o) for o in oraciones_etiquetadas]

#### Ahora podemos lematizar cada oración del texto con la ayuda de un diccionario de lemas.

In [10]:
def lematizar_oracion(oracion, lemas):
    oracion_tokenizada = []
    for t in oracion:
        temp = t[0] + " " + t[1]
        if (temp in lemas):
            oracion_tokenizada.append((lemas[temp], t[1]))
        else:
            oracion_tokenizada.append(t)
    return oracion_tokenizada
lemas = cargar_archivo_pickle("lemmas_dict")
oraciones_etiquetadas = [lematizar_oracion(o, lemas) for o in oraciones_etiquetadas]

#### Ahora obtenemos el vocabulario para proceder a encontrar la entropía condicional de alguna palabra dada

In [11]:
import functools
import operator
vocabulario = functools.reduce(operator.concat, oraciones_etiquetadas)
vocabulario = set(vocabulario)
print("longitud del vocabulario:", len(vocabulario))

longitud del vocabulario: 6798


#### Ahora que tenemos las oraciones normalizadas podemos empezar a buscar la entropía condicional de una palabra en relación con todas las demás:

In [12]:
def entropia_condicional(palabra, oraciones, vocabulario):
    import numpy as np
    n = len(oraciones)
    entropias = []
    # probabilidad que palabra1 = 1
    palabra_count = 0
    for o in oraciones:
        if palabra in o:
            palabra_count += 1
    p1_1 = palabra_count / n
    p1_0 = 1 - p1_1
    for p in vocabulario:
        # probabilidad de que palabra2 = 1
        # y probabilidad conjunta
        palabra_count2 = 0
        palabra_count_ambas = 0
        for o in oraciones:
            if p in o:
                palabra_count2 += 1
                if palabra in o:
                    palabra_count_ambas += 1
        p2_1 = palabra_count2 / n
        p2_0 = 1 - p2_1
        p_1_1 = palabra_count_ambas / n
        p_0_1 = p2_1 - p_1_1
        p_0_0 = p1_0 - p_0_1
        p_1_0 = p2_0 - p_0_0
        if (p_1_1 == 0):
            continue
        # calculo de la entropía
        e_0_0 = 0
        e_1_0 = 0
        e_0_1 = 0
        e_1_1 = 0
        if (p_0_0 != 0):
            e_0_0 = p_0_0 * np.log(p_0_0 / p2_0)
        if (p_1_0 != 0):
            e_1_0 = p_1_0 * np.log(p_1_0 / p2_0)
        if (p_0_1 != 0):
            e_0_1 = p_0_1 * np.log(p_0_1 / p2_1)
        if (p_1_1 != 0):
            e_1_1 = p_1_1 * np.log(p_1_1 / p2_1)
        ent = -1 * (e_0_0 + e_1_0 + e_0_1 + e_1_1)
        entropias.append((p, ent))
        entropias.sort(key = lambda x: x[1])
    return entropias
palabra = ("sindicato", "n")
ent = entropia_condicional(palabra, oraciones_etiquetadas, vocabulario)
for e in ent:
    print(e)

(('sindicato', 'n'), -0.0)
(('trabajadores', 's'), 0.057820908799272965)
(('sutaur', 's'), 0.06225641762691381)
(('sindicalizados', 'n'), 0.06252388992374401)
(('afiliados', 'n'), 0.06252388992374401)
(('dirigencia', 'n'), 0.06269320975330067)
(('hernández', 'n'), 0.06385321730566412)
(('liquidarse', 's'), 0.06388155147243174)
(('secretas', 'n'), 0.06388155147243174)
(('lado', 'n'), 0.06442258968614199)
(('llegar', 'v'), 0.06470934129621467)
(('barco', 'n'), 0.06490617775759797)
(('trabajador', 'n'), 0.06493640093777031)
(('negociación', 'n'), 0.06498935028023523)
(('galván', 'n'), 0.065222679922835)
(('dirigente', 'n'), 0.06542488292224895)
(('garcía', 'n'), 0.06547821433572698)
(('oferta', 'n'), 0.06547821433572698)
(('gobierno', 'n'), 0.06554358800174992)
(('francisco', 'n'), 0.06569224914994708)
(('establecer', 'v'), 0.06569224914994708)
(('ayer', 'r'), 0.06569487514992396)
(('unico', 'n'), 0.06569879483855139)
(('gastarse', 's'), 0.06569879483855139)
(('proteger', 'v'), 0.06569879